In [1]:
import sys
sys.path.append('../')

import tquant as tq


import tensorflow as tf
from datastore import *
import matplotlib.pyplot as plt
import QuantLib as ql

def get_ql_curve(curve_tq, evaluation_date):
    evaluation_date_ql = ql.Date.from_date(evaluation_date)
    dates_ql = [evaluation_date_ql + d for d in curve_tq._pillar_days]
    rates_ql = [r.numpy() for r in curve_tq._rates]
    curve_ql = ql.ZeroCurve(dates_ql, rates_ql, ql.ActualActual(ql.ActualActual.ISDA), ql.TARGET())
    curve_ql.enableExtrapolation()
    return curve_ql


# Settings

In [3]:
tq.Settings.evaluation_date = date(2024, 4, 30)

calendar = tq.TARGET()
daycounter = tq.DayCounter(tq.DayCounterConvention.Actual360)
evaluation_date = tq.Settings.evaluation_date

rates = ir_eur_curve_estr['quote'].values
times = ir_eur_curve_estr['daycount'].values/365


rate_curve = tq.RateCurve(evaluation_date, times, rates, 'LINEAR')
eur6m_index = tq.IborIndex(calendar, 6, tq.TimeUnit.Months, tq.Currency.EUR)

# Test QL

In [4]:
a = 0.03
sigma = 0.2
#TQ
hw_tq = tq.HullWhiteProcess(rate_curve, a, sigma)
# QL
rate_curve_ql = get_ql_curve(rate_curve, evaluation_date)
hrate_curve_ql = ql.YieldTermStructureHandle(rate_curve_ql)
hw_ql = ql.HullWhiteProcess(hrate_curve_ql, a, sigma)


In [5]:
t0 = 2
x0 = 0.01
dt = 3.3

print("tq drift: ", hw_tq.expectation(x0=x0, dt=dt, t0=t0).numpy() )
print("ql drift: ", hw_ql.expectation(t0, x0, dt) ) # differenza data dall'inst. forward
print("*"*30)
print("tq diffusion: ", hw_tq.std_deviation(x0=x0, dt=dt, t0=t0).numpy() )
print("ql diffusion: ", hw_ql.stdDeviation(t0, x0, dt) )

tq drift:  0.42119516645478117
ql drift:  0.421211266538131
******************************
tq diffusion:  0.34605408716188574
ql diffusion:  0.34605408716188574


In [6]:
dw = np.random.normal()
print("QL evolve: ", hw_ql.evolve(t0, x0, dt, dw))
print("TQ evolve: ", hw_tq.evolve(t0, x0, dt, dw).numpy())

QL evolve:  0.3059935613534811
TQ evolve:  0.30597746127013126


In [8]:
process = tq.HullWhiteProcess(rate_curve, a, sigma)
time_grid = np.linspace(start=0.0, stop=10, num=60, retstep=False)
kernel = tq.GaussianPathGenerator(hw_tq, time_grid)

In [9]:
kernel.simulate(n_paths=100000)

<tf.Tensor: shape=(100000, 60), dtype=float64, numpy=
array([[ 3.90613283e-02,  1.38530208e-01,  1.07311986e-01, ...,
         1.63054088e+00,  1.49220753e+00,  1.45691688e+00],
       [ 3.90613283e-02, -3.73553000e-02,  7.30364440e-02, ...,
         1.30885836e+00,  1.52537914e+00,  1.51146786e+00],
       [ 3.90613283e-02,  6.46984086e-02,  1.42910148e-01, ...,
         1.63688252e+00,  1.70581935e+00,  1.65443594e+00],
       ...,
       [ 3.90613283e-02, -5.36651148e-02,  8.68353670e-03, ...,
         9.34250369e-01,  9.78699995e-01,  9.93115682e-01],
       [ 3.90613283e-02, -1.52429158e-02,  1.75886315e-03, ...,
         1.33646827e+00,  1.29399076e+00,  1.26651726e+00],
       [ 3.90613283e-02,  4.86743070e-02, -2.36146749e-02, ...,
         1.89760432e+00,  1.97315824e+00,  2.05329567e+00]])>

In [10]:
# creazione curve future zcb vs ql
# pricing swap a date future

## QL

In [10]:
evaluation_date_ql = ql.Date.from_date(evaluation_date)
simulation_end = evaluation_date_ql + ql.Period("20Y")
gridStepPeriod = ql.Period(3, ql.Months)
timeGrid = Grid(evaluation_date_ql, simulation_end, gridStepPeriod)
times = np.array(timeGrid.GetTimes())
dates = np.array(timeGrid.GetDates())

process = hw_ql
Npaths = 100

timestep_len= len(times[1:])
dimension = process.factors()

rng = ql.UniformRandomSequenceGenerator(dimension * timestep_len, ql.UniformRandomGenerator())
sequenceGenerator = ql.GaussianRandomSequenceGenerator(rng)
pathGenerator = ql.GaussianMultiPathGenerator(process, list(times), sequenceGenerator, False)

# singolo path QL-like
generator = sequenceGenerator

sequence = generator.nextSequence().value() # per la timegrid data
m = process.size()
n = process.factors()
asset = list(process.initialValues())
time_grid = times
dt = times[1:]-times[:-1]

path = np.zeros((m, len(time_grid)))
for j in range(m):
    path[j,0] = asset[j]

for i in range(1, path.size):
    offset = (i-1)/n
    offset_int = int(offset)
    temp = np.array(sequence[offset_int:offset_int+n])
    pass
    path[j,i] = process.evolve(time_grid[i], asset[0], dt[i-1], temp[0])
    
# multipli path QL-like
gen_paths = np.zeros((dimension, timestep_len+1, Npaths))
for i in range(Npaths):
    samplePath = pathGenerator.next()
    values = samplePath.value() 

    for j in range(dimension):
        gen_paths[j, :, i] = values[0]

NameError: name 'Grid' is not defined

In [ ]:
# implementazione tq


# OLD

In [ ]:
hw = tq.HullWhiteProcess(a, sigma, rate_curve)
hw_kernel = tq.GaussianRateKernel1D(hw)

# simulate short rate and discount curve 
paths = int(1e6) 
r_tensor, zb_tensor, grid = hw_kernel.rate_tensor(paths,
                                                  last_grid_time=1,
                                                  time_steps=2) 
# Reshape the tensor and create a discount curve object
reshaped_tensor = tf.reshape(zb_tensor, shape=(zb_tensor.shape[1], paths, 1))
rate_curve_tensor = tq.RateCurve.from_zcb(evaluation_date, times, reshaped_tensor, "LINEAR")

In [ ]:
rate_curve_tensor.discount(2.0)

<tf.Tensor: shape=(1000000, 1), dtype=float64, numpy=
array([[0.99560683],
       [1.00017447],
       [0.98124546],
       ...,
       [0.99541913],
       [0.99054788],
       [0.99026692]])>